In [1]:
import torch.nn as nn
from model_numba.Layers.Layers import batchNorm2D, Convolution2D_GPU
import numpy as np
from torchvision import transforms
from model_numba.unet_cuda import Double_Conv, Unet_Cuda
from PIL import Image
import torch
import time
from numba import cuda

In [2]:
transform = transforms.Compose(
    [
        transforms.Resize((512, 512)),
        transforms.ToTensor()
    ]
)
image = Image.open('test.jpg')
print(image.size)
image = transform(image).unsqueeze(0).repeat(4, 1, 1, 1)
image.shape

(500, 300)


torch.Size([4, 3, 512, 512])

In [3]:
from model.unet import U_net
model = U_net(1).eval()
weight = torch.load('weights/weights.pth')
model.load_state_dict(weight)

numba_model = Unet_Cuda(1).eval()

In [4]:

start = time.time()
out = model(image)
end = time.time()

print(f'Torch Runtime: {end- start} s')

start = time.time()
out_numba = numba_model(image.cpu().detach().numpy())
end = time.time()

print(f'Numba Runtime: {end- start} s')

mean = np.mean(np.abs(model.d5.cpu().detach().numpy()- numba_model.d5.copy_to_host()))
max = np.max(np.abs(model.d5.cpu().detach().numpy()- numba_model.d5.copy_to_host()))
print(f'Mean Difference: {mean : .6f}')
print(f'Max Difference: {max: .6f}')

Torch Runtime: 3.5811500549316406 s


CudaAPIError: [2] Call to cuMemAlloc results in CUDA_ERROR_OUT_OF_MEMORY

In [ ]:
mean = np.mean(np.abs(model.u1.cpu().detach().numpy() - numba_model.u1.copy_to_host()))
max = np.max(np.abs(model.u1.cpu().detach().numpy() - numba_model.u1.copy_to_host()))
print(f'Mean Difference: {mean : .6f}')
print(f'Max Difference: {max: .6f}')

Mean Difference:  0.000014
Max Difference:  0.000922


In [ ]:
mean = np.mean(np.abs(model.u2.cpu().detach().numpy() - numba_model.u2.copy_to_host()))
max = np.max(np.abs(model.u2.cpu().detach().numpy() - numba_model.u2.copy_to_host()))
print(f'Mean Difference: {mean : .6f}')
print(f'Max Difference: {max: .6f}')

Mean Difference:  0.000016
Max Difference:  0.000747


In [ ]:
mean = np.mean(np.abs(model.u3.cpu().detach().numpy() - numba_model.u3.copy_to_host()))
max = np.max(np.abs(model.u3.cpu().detach().numpy() - numba_model.u3.copy_to_host()))
print(f'Mean Difference: {mean : .6f}')
print(f'Max Difference: {max: .6f}')

Mean Difference:  0.000016
Max Difference:  0.000887


In [ ]:
mean = np.mean(np.abs(model.u4.cpu().detach().numpy() - numba_model.u4.copy_to_host()))
max = np.max(np.abs(model.u4.cpu().detach().numpy() - numba_model.u4.copy_to_host()))
print(f'Mean Difference: {mean : .6f}')
print(f'Max Difference: {max: .6f}')

Mean Difference:  0.000028
Max Difference:  0.000770


In [ ]:
mean = np.mean(np.abs(out.cpu().detach().numpy()- out_numba))
max = np.max(np.abs(out.cpu().detach().numpy()- out_numba))
print(f'Mean Difference: {mean : .6f}')
print(f'Max Difference: {max: .6f}')

Mean Difference:  0.000504
Max Difference:  0.011502
